# Exercise 1: Policy Iteration

In [ ]:
# The autoreload extension will automatically load in new code as you edit files, 
# so you don't need to restart the kernel every time
%load_ext autoreload
%autoreload 2
from typing import Tuple, List
import numpy as np
from scipy.special import softmax
from ipywidgets import interact

from grid_world import GridWorld

In this exercise we will be using a `GridWorld` environment where there are a set of absorbing states that get a reward of 0 and for every other state visited gets a reward of -1. The actions are to go up, down, left, or right, respecting the boundaries of the grid. We can visualize the grid below.

In [ ]:
env = GridWorld(4, absorbing_states={(0,0), (3,3)})
env.render();

#### Exercise 1.1: Implement Policy Evaluation
First, implement the function `bellman_expectation` which will compute the one-step Bellman update of the value function for a given state and stochastic policy. Then, implement the function `policy_evaluation` to implement the policy evaluation algorithm using the `bellman_expectation` function.

In [ ]:
def compute_action_value(env: GridWorld, state: Tuple[int, int], action: Tuple[int, int], discount: float):
    """
    Compute the action value Q(x, u) for current state and action using one-step lookahead with the
    environment's value function.

    Args:
        env: grid world environment
        state: the (x, y) indices that define the location in the grid
        action: the movement direction (i.e. (1, 0)) action
        discount: discount factor
    """
    next_state, reward = env.step(state, action)
    return reward + discount * env.state_value[next_state]

def bellman_expectation(env: GridWorld, state: Tuple[int, int], policy_probs: List[float], discount: float) -> float:
    """
    Performs a one-step lookahead and applies the Bellman expectation equation to update the value for
    the given state.
    
    Args:
        env: grid world environment
        state: the (x, y) indices that define the location in the grid
        policy_probs: transition probabilities for each action for the given state
        discount: discount factor
    Returns:
        the new value for the specified state
    """
    ##### YOUR CODE STARTS HERE #####
    # Hint: Use env.actions.items() and compute_action_value(...)

    ###### YOUR CODE END HERE ######

def policy_evaluation(env: GridWorld, policy: np.ndarray, eps: float= 1e-2, max_steps: int=1, discount: float=1., in_place: bool=False):
    """
    Args:
        env: grid world environment
        policy: a numpy 3-D numpy array, where the first two dimensions identify a state and 
                the third dimension identifies the actions. The array stores the probability 
                of taking each action. I.e. policy[i, j, k] is the probability of selection action
                k from state (i,j).
        eps: convergence threshold
        max_steps: max number of iterations of the algorithm
        discount: discount factor
        in_place: if False, the value table is updated after all the new values have been calculated.
             if True the state [i, j] will new already new values for the states [< i, < j]
    """
    state_value_history = [env.state_value]
    converged = False
    for k in range(max_steps):
        values_prev = np.copy(env.state_value)
        # cache old values if not in place
        values = env.state_value if in_place else np.empty_like(
            env.state_value)
        ##### YOUR CODE STARTS HERE #####

        ###### YOUR CODE END HERE ######
        # set the new value table
        env.state_value = values
        state_value_history.append(env.state_value)
        if (np.linalg.norm(values_prev - values, ord=np.inf) < eps):
            converged = True
            break
    return converged, state_value_history

Now, run the code below to run the policy evaluation algorithm on a simple grid world using a default random policy. Try playing around with a different discount factor to see how that effects the solution.

In [ ]:
# Create GridWorld
env = GridWorld(4, absorbing_states={(0,0), (3,3)})

# Define random policy
policy = np.ones((*env.state_value.shape, len(env.actions))) * 0.25

# Run policy evaluation
num_steps = 10
discount = 1.0
converged, state_value_history = policy_evaluation(env, policy, max_steps=num_steps, discount=discount, in_place=False)
print(f"Converged: {converged}")
@interact(i=(0, num_steps))
def f(i=0):
    env.render(state_value_history[i]);

Try playing around with an even larger example world below.

In [ ]:
large_env = GridWorld(7, absorbing_states={(0,0), (6,6)})
policy = np.ones((*large_env.state_value.shape, len(env.actions))) * 0.25
converged, _ = policy_evaluation(large_env, policy, max_steps=1000, discount=1., in_place=False)
print(f"Converged: {converged}")
large_env.render(policy=policy);

#### Exercise 1.2: Implement Policy Iteration
Now, implement the `policy_iteration` algorithm below, leveraging the `policy_evaluation` function you implemented above.

In [ ]:
def policy_iteration(env: GridWorld, eps: float= 1e-2, max_steps: int=1, discount: float=1., β: float=1., in_place: bool=False):
    """
    Args:
        env: grid world environment
        eps: convergence threshold
        max_steps: max number of iterations of the algorithm
        discount: discount factor
        β: Boltzmann constant for softmax for stochastic policy definition from action values
        in_place: if False, the value table is updated after all the new values have been calculated.
             if True the state [i, j] will new already new values for the states [< i, < j]

    Returns:
        policy: a numpy 3-D numpy array, where the first two dimensions identify a state and 
                the third dimension identifies the actions. The array stores the probability 
                of taking each action. I.e. policy[i, j, k] is the probability of selection action
                k from state (i,j).
    """
    converged = False
    # Reset environment state value function and initialize policy with uniform distribution
    env.reset()
    policy = np.ones((*env.state_value.shape, len(env.actions))) * 0.25
    for k in range(max_steps):
        policy_prev = np.copy(policy)
        ##### YOUR CODE STARTS HERE #####
        # Hint: use softmax() to compute the new policy probabilities
        # Update state value function, env.state_value

        ###### YOUR CODE END HERE ######
        
        if (np.linalg.norm((policy - policy_prev).flatten(), ord=np.inf) < eps):
            converged = True
            break   
    return converged, policy

Now, we will revisit the large grid world and see how the optimal value function is different from the value function for the random policy evaluated above.

In [ ]:
large_env = GridWorld(7, absorbing_states={(0,0), (6,6)})
converged, policy = policy_iteration(large_env, max_steps=1000, discount=1., β=5.)
print(f"Converged: {converged}")
large_env.render(policy=policy);